# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import SparkSession

# Crear la sesión Spark
spark = SparkSession.builder \
    .appName("Airbnb") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
    
# Leer el archivo CSV de precios
airbnb_price = spark.read.option("sep", ",") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .csv("dbfs:/FileStore/tables/airbnb_price.csv")

# Leer el archivo TSV de últimas revisiones
airbnb_last_review = spark.read.option("sep", "\t") \
                               .option("header", "true") \
                               .option("inferSchema", "true") \
                               .csv("dbfs:/FileStore/tables/airbnb_last_review.tsv")

airbnb_room_type = spark.read.option("sep", ";") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("dbfs:/FileStore/tables/airbnb_room.csv") \
                              

airbnb_room_type.show(3)
airbnb_price.show(3)
airbnb_last_review.show(3)

df_airbnb = airbnb_price \
    .join(airbnb_last_review, "listing_id", "inner") \
    .join(airbnb_room_type, "listing_id", "inner")

df_airbnb.display()


+----------+--------------------+---------------+
|listing_id|         description|      room_type|
+----------+--------------------+---------------+
|      2595|Skylit Midtown Ca...|Entire home/apt|
|      3831|Cozy Entire Floor...|Entire home/apt|
|      5099|Large Cozy 1 BR A...|Entire home/apt|
+----------+--------------------+---------------+
only showing top 3 rows

+----------+-----------+--------------------+
|listing_id|      price|         nbhood_full|
+----------+-----------+--------------------+
|      2595|225 dollars|  Manhattan, Midtown|
|      3831| 89 dollars|Brooklyn, Clinton...|
|      5099|200 dollars|Manhattan, Murray...|
+----------+-----------+--------------------+
only showing top 3 rows

+----------+-----------+------------+
|listing_id|  host_name| last_review|
+----------+-----------+------------+
|      2595|   Jennifer| May 21 2019|
|      3831|LisaRoxanne|July 05 2019|
|      5099|      Chris|June 22 2019|
+----------+-----------+------------+
only showing

listing_id,price,nbhood_full,host_name,last_review,description,room_type
2595,225 dollars,"Manhattan, Midtown",Jennifer,May 21 2019,Skylit Midtown Castle,Entire home/apt
3831,89 dollars,"Brooklyn, Clinton Hill",LisaRoxanne,July 05 2019,Cozy Entire Floor of Brownstone,Entire home/apt
5099,200 dollars,"Manhattan, Murray Hill",Chris,June 22 2019,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
5178,79 dollars,"Manhattan, Hell's Kitchen",Shunichi,June 24 2019,Large Furnished Room Near B'way,private room
5238,150 dollars,"Manhattan, Chinatown",Ben,June 09 2019,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt
5295,135 dollars,"Manhattan, Upper West Side",Lena,June 22 2019,Beautiful 1br on Upper West Side,Entire home/apt
5441,85 dollars,"Manhattan, Hell's Kitchen",Kate,June 23 2019,Central Manhattan/near Broadway,Private room
5803,89 dollars,"Brooklyn, South Slope",Laurie,June 24 2019,"Lovely Room 1, Garden, Best Area, Legal rental",Private room
6021,85 dollars,"Manhattan, Upper West Side",Claudio,July 05 2019,Wonderful Guest Bedroom in Manhattan for SINGLES,Private room
6848,140 dollars,"Brooklyn, Williamsburg",Allen & Irina,June 29 2019,Only 2 stops to Manhattan studio,entire home/apt


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, min, max

# Convertir la columna 'last_review' a tipo de dato fecha usando un formato más estándar
df_date = df_airbnb.withColumn("last_review_date", to_date(col("last_review"), "MMM dd yyyy"))

# Obtener la primera y última reseña
first_review = df_date.agg(min("last_review_date")).collect()[0][0]
last_review = df_date.agg(max("last_review_date")).collect()[0][0]

# Imprimir los resultados
print("Primera reseña:", first_review)
print("Última reseña:", last_review)

Primera reseña: 2019-01-01
Última reseña: 2019-07-09


In [0]:
from pyspark.sql.functions import lower

df_airbnb_lower = df_airbnb.withColumn("room_type", lower(col("room_type")))

private_rooms_count = df_airbnb_lower.filter(df_airbnb_lower['room_type'] == "private room").count()

print(f"Cantidad de anuncios con habitación privada: {private_rooms_count}")

Cantidad de anuncios con habitación privada: 11282


In [0]:
from pyspark.sql.functions import split, avg

df_price = df_airbnb.withColumn(
    'price',
    split(col('price'), ' ')[0]
)

avg_price = df_price.agg(avg('price')).collect()[0][0]

avg_price_rounded = round(avg_price, 2)

print(f"El precio promedio de los anuncios es: ${avg_price_rounded}")

El precio promedio de los anuncios es: $141.78


In [0]:
from pyspark.sql import Row

review_dates = spark.createDataFrame([
    Row(first_reviewed=first_review, last_reviewed=last_review, nb_private_rooms=private_rooms_count, avg_price=avg_price_rounded)
])

review_dates.display()

first_reviewed,last_reviewed,nb_private_rooms,avg_price
2019-01-01,2019-07-09,11282,141.78
